In [6]:
import numpy as np
import networkx as nx
from collections import defaultdict

from QHyper.converter import Converter
from QHyper.constraint import Polynomial

from dwave.system import DWaveSampler
from dimod import BinaryQuadraticModel

from QHyper.problems.community_detection import CommunityDetectionProblem, Network

Some util function from QHyper

In [3]:
def convert_qubo_keys(qubo: Polynomial) -> tuple[dict[tuple, float], float]:
    new_qubo = defaultdict(float)
    offset = 0.0

    qubo, offset = qubo.separate_const()
    for k, v in qubo.terms.items():
        if len(k) == 1:
            new_key = (k[0], k[0])
        elif len(k) > 2:
            raise ValueError("Only supports quadratic model")
        else:
            new_key = k

        new_qubo[new_key] += v

    return (new_qubo, offset)

Load our biggest graph

In [4]:
Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)
G = Graphs[9]

#### First let's find clique embedding for graph of 100 nodes
We'll be checking cache'ing behaviour: https://docs.ocean.dwavesys.com/en/stable/docs_minorminer/source/reference/clique_embedding.html#minorminer.busclique.busgraph_cache

In [8]:
sampler = DWaveSampler(solver="Advantage_system5.4", region="eu-central-1")

In [9]:
network = Network(graph=G)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)

In [10]:
bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_59808\3242365722.py:1: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [11]:
from dwave.embedding.pegasus import find_clique_embedding

Let's clear caches for safety

In [13]:
from minorminer.busclique import busgraph_cache


busgraph_cache.clear_all_caches()

In [14]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 1min 44s
Wall time: 2min 19s


In [20]:
len(emb.keys())

100

In [21]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 0 ns
Wall time: 69 ms


In [22]:
from dwave.system.composites import FixedEmbeddingComposite


sample_cl = FixedEmbeddingComposite(sampler, embedding=emb).sample(bqm, num_reads=100)

res_cl = sample_cl.first.sample

c0_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 0])
c1_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 1])

print("c0_cl len:", len(c0_cl))
print("c1_cl len:", len(c1_cl))

print("Sample modularity from FixedEmbeddingComposite:", nx.community.modularity(G, [c0_cl, c1_cl]))

c0_cl len: 53
c1_cl len: 47
Sample modularity from FixedEmbeddingComposite: 0.4785225997347209


Sampling works.

#### Now let's take a subcommunity from that graph, subcommunity of 99 nodes:

In [23]:
network = Network(graph=G, community=[*range(99)])
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)

bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_59808\176734912.py:8: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [24]:
%time emb_2 = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 46.9 ms
Wall time: 80.5 ms


In [25]:
len(emb_2.keys())

99

As we can see from the time measured, the cached embedding was used to find embedding for these 99 nodes.

Now let's see the cache'ing behaviour of the cache'ing for a smaller graph (different graph, not subcommunity):

In [30]:
G_90 = Graphs[8]

network = Network(graph=G_90)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)
bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_59808\3945948250.py:9: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [31]:
%time emb_3 = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 31.2 ms
Wall time: 73.3 ms


It used cached embedding as well.

Ok, so now let's take a bigger graph, new one, of 150 nodes.

In [32]:
G_150 = nx.powerlaw_cluster_graph(n=150, m=2, p=0.2)
network = Network(graph=G_150)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)
bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_59808\1647688498.py:8: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [33]:
%time emb_150 = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 46.9 ms
Wall time: 75.1 ms


It used cache as well.

In [35]:
len(emb_150.keys())

150

That's because find_clique_embedding uses busgraph_cache(target_graph).find_clique_embedding(nn) inside: \
https://docs.ocean.dwavesys.com/en/stable/docs_minorminer/source/reference/generated/minorminer.busclique.busgraph_cache.find_clique_embedding.html#minorminer.busclique.busgraph_cache.find_clique_embedding

In [53]:
emb_cached_150 = busgraph_cache(sampler_graph).find_clique_embedding(['x'+str(i) for i in range(150)])

In [56]:
emb_cached_150 == emb_150

True

So the embeddings are cached up to the size of:

In [36]:
busgraph_cache(sampler_graph).largest_clique().keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168])

In [37]:
len(busgraph_cache(sampler_graph).largest_clique().keys())

169

#### Trying to extract info from the caching files

In [72]:
import os
from minorminer.busclique import busgraph_cache


# From docs - this is where they are stored
cache_dir = busgraph_cache.cache_rootdir()
print(f"Cache directory: {cache_dir}")

cache_files = os.listdir(cache_dir)
print(f"Cache files in {busgraph_cache.cache_rootdir()}:", cache_files)

# Inside the "clique" dir
cache_dir += "\\clique"
cache_files = os.listdir(cache_dir)
print(f"Cache files in {cache_dir}:", cache_files)

Cache directory: C:\Users\basia\AppData\Local\dwave\busclique_7
Cache files in C:\Users\basia\AppData\Local\dwave\busclique_7: ['clique']
Cache files in C:\Users\basia\AppData\Local\dwave\busclique_7\clique: ['.lock', '.lru', 'pegasus_599a4b13e9291929']


In [84]:
import pickle


lru_file_path = os.path.join(cache_dir, '.lock')
print(lru_file_path)


with open(lru_file_path, 'rb') as file:
    cache_content = pickle.load(file)


print(type(cache_content))
print("Cache content:", cache_content)

C:\Users\basia\AppData\Local\dwave\busclique_7\clique\.lock


EOFError: Ran out of input

Oops...

In [85]:
lru_file_path = os.path.join(cache_dir, '.lru')
print(lru_file_path)


with open(lru_file_path, 'rb') as file:
    cache_content = pickle.load(file)

print(type(cache_content))
print("Cache content:", cache_content)

C:\Users\basia\AppData\Local\dwave\busclique_7\clique\.lru
<class 'list'>
Cache content: ['pegasus_599a4b13e9291929']


In [86]:
%%capture cap


lru_file_path = os.path.join(cache_dir, 'pegasus_599a4b13e9291929')
print(lru_file_path)


with open(lru_file_path, 'rb') as file:
    cache_content = pickle.load(file)

print("Cache content:", cache_content)

with open('cache_file.txt', 'w') as file:
    file.write(cap.stdout)

print(type(cache_content))